# Alumno: Gerardo de Miguel González

## Minería de Datos (Master en Data Science, UIMP-UC)  2018-2019

## Profesores: Sixto Herrera y Rodrigo García

## T01 Variables Categóricas: Reglas de Asociación y Árboles de Clasificación

En la presente tarea consideraremos el dataset `Mushroom`, incluido tanto en la La librería [arulesViz](https://cran.r-project.org/web/packages/arulesViz/arulesViz.pdf) como en las diferentes plataformas descritas en el marco de la asignatura y en el GitHub dedicado a este Máster ([Mushroom](https://github.com/SantanderMetGroup/Master-Data-Science/blob/master/Data_mining/datasets/mushrooms.csv.)), para aplicar las diferentes técnicas vistas en el curso para variables categóricas: Reglas de Asociación y Árboles de Clasificación.

Para el desarrollo de la tarea se permitirá el uso de todo el material incluido en el Moodle de las asignatura así como el desarrollado por el alumno durante la realización de las prácticas.

La entrega consisitirá de un notebook de Jupyter ó un R-MarkDown, junto con el archivo html que éste genera. Ambos ficheros se entregarán a través del Moodle de la asignatura en la tarea correspondiente.

### ::GMG::Punto 0.

#### Bibliotecas

In [ ]:
#::GMG::El primer paso es cargar las bibliotecas necesarias pra hace la práctica
library(RCurl) #::GMG::Get remote dataset

#### Carga de datos

In [ ]:
#::GMG::También necesitamos el dataset
#       http://archive.ics.uci.edu/ml/datasets/Mushroom
# specify the URL for the Iris data CSV
urlfile <-'http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
downloaded <- getURL(urlfile, ssl.verifypeer=FALSE)
connection <- textConnection(downloaded)
dataset <- read.csv(connection, header=FALSE)

#### Exploración  y limpieza 

In [ ]:
#::GMG::Vemos lo que hemos descargado
head(dataset)

In [ ]:
#::GMG::Hay que poner nombres a las columnas
#::nota:: LEER
# http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
#::nota::no se pueden poner '-' en los nombres de las columnas en R por conflicto con operador "-"
names(dataset) = c('class','cap.shape','cap.surface','cap.color','bruises','odor',
                   'gill.attachment','gill.spacing','gill.size','gill.color','stalk.shape',
                   'stalk.root','stalk.surface.above.ring', 'stalk.surface.below.ring',
                   'stalk.color.above.ring','stalk.color.below.ring','veil.type',
                   'veil.color','ring.number','ring.type','spore.print.color',
                   'population','habitat')
head(dataset)

In [ ]:
#::GMG::Resultado que obtenemos
str(dataset)

In [ ]:
#::Tamaño del dataset
dim(dataset)

In [ ]:
#::GMG::Proporción de clases en el dataset
table(dataset$class)
barplot(height = table(dataset$class),
        col = c('blue','red'),
        names.arg = c('edible','poisonous'))

In [ ]:
#::GMG::Dos observaciones
#::nota 1:: 8. Missing Attribute Values: 2480 of them (denoted by "?"), all for
#         attribute #11 (stalk-root)
# en http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
#
summary(dataset$stalk.root)

In [ ]:
#::nota 2::veil-type solamente tiene un valor 'p'
#
# veil_type_idx <- which(colnames(dataset) == "veil-type")
# dataset <- dataset[-veil_type_idx]
summary(dataset$veil.type)

In [ ]:
#::GMG::Nos cargamos el feature que vale siempre lo mismo y ni aporta
#       nada, nos quedamos con 22 factores
dataset_complete <- dataset
dataset_complete$veil.type <- NULL

In [ ]:
#::GMG::Anotamos de forma apropiada los missing values
dataset_complete$stalk.root[dataset_complete$stalk.root == '?'] <- NA
#::GMG::Me deshago el level '?' en stalk_root
# https://www.rdocumentation.org/packages/base/versions/3.5.1/topics/droplevels
dataset_complete$stalk.root<-droplevels(dataset_complete$stalk.root)
summary(dataset_complete$stalk.root)

In [ ]:
str(dataset_complete)

#### Missing Values

**::GMG::** Hay que decidir qué hacer con la característica (*feature*) `stalk-root` que tiene un 31% de los niveles `NA`. Hay dos caminos que puedo seguir:

1. Eliminar la feature
2. Usar un mecanismo de imputación

Elijo el camino 1 siendo consciente de que la eliminación de esta *feature* no es como el caso de `veil-type` y elimino una fuente de información para la clasificación.

In [ ]:
dataset_complete$stalk.root <- NULL

In [ ]:
#::GMG::Ya tenemos el dataset que vamos a usar
str(dataset_complete)

In [ ]:
#::GMG::Balance de clases en el dataset
table(dataset_complete$class)

## A. Reglas de Asociación

### Punto 1 (3 puntos):

Considerar uno de los algoritmos de asociación vistos en clase y obtener las reglas representativas del dataset fijando los parámetros de aprendizaje (soporte, confianza, etc...). 

#### Previo

[Hashler et al. arulesViz](https://cran.r-project.org/web/packages/arulesViz/vignettes/arulesViz.pdf) pp.3

> Before we start, we set the number of displayed significant digits to two to make the output
> easier to read, and we set the seed for the random number generator for predictability.

In [ ]:
options(digits = 2)
set.seed(1234)

#### Bibliotecas 

In [ ]:
library("arules")

In [ ]:
library("arulesViz")

#### Transacciones

In [ ]:
#::GMG::Creamos las transacciones de base "desmenuzando" las "features" en "items"
#       con los niveles de sus factores
transactions.ms <- as(dataset_complete, 'transactions')
transactions.ms

In [ ]:
summary(transactions.ms)

#### Apriori

**::GMG::** Elijo como algoritmo de clasificación: `A Priori`

https://www.rdocumentation.org/packages/arules/versions/1.6-2/topics/apriori

In [ ]:
rules.ap <- apriori(data = transactions.ms, 
                    parameter=list(support = 0.1, confidence = 0.8,
                                   minlen = 2, maxlen = 10),
                    appearance=list(rhs = c('class=e'), default = 'lhs')
                   )

#### Análisis General

Analizar los resultados en términos generales:

* ¿Cuantas reglas se han generado?

In [ ]:
rules.ap

In [ ]:
summary(rules.ap)

* ¿Existe alguna regla redundante?, ¿Cuántas?

In [ ]:
#::GMG::Según el ejercicio S04_Association_Rules hecho en clase
length(rules.ap[is.redundant(rules.ap)])

In [ ]:
#::GMG::Filtramos las redundantes
rules.ap.filtered <- rules.ap[!is.redundant(rules.ap)]

In [ ]:
length(rules.ap.filtered)

In [ ]:
summary(rules.ap.filtered)

In [ ]:
inspect(head(rules.ap.filtered,n = 25, by = 'lift'))

* ¿Existe alguna regla que incluya la variable objetivo: `Class=edible` ó `Class=poisonous`?, ¿Cuantas?

In [ ]:
#::GMG::Para eso hay que usar inspect()/length() + subset()
length(subset(rules.ap.filtered, 
              subset = (rhs %in% c('class=e') | lhs %in% c('class=e')) |
                       (rhs %in% c('class=p') | lhs %in% c('class=p'))
             )
      )

* De cara a ser utilizada como modelo predictivo es adecuado que la variable objetivo se encuentre en el consecuente de la regla de asociación, ¿se da esta propiedad en alguna regla?

In [ ]:
#::GMG::Queremos que class se encuentre en rhs, claro :)
rules.ap.class <- subset(rules.ap.filtered, 
                         subset = rhs %in% c('class=e') | rhs %in% c('class=p'))

In [ ]:
#::GMG::En total
length(rules.ap.class)

In [ ]:
#::GMG::Reglas de comestibles
length(subset(rules.ap.class, subset = rhs %in% c("class=e")))

In [ ]:
#::GMG::Reglas de venenosas
length(subset(rules.ap.class, subset = rhs %in% c("class=p")))

In [ ]:
summary(rules.ap.class)

#### Análisis Detallado

* Considerar los subconjuntos de reglas con ambas clases como consecuente e ilustrar las variables implicadas en cada caso. Considerar alguno de los grafos vistos para apoyar las conclusiones obtenidas.

In [ ]:
#::GMG::Ejemplo de las reglas de comestibles con más "lift"
rules.ap.class.edible <- subset(rules.ap.class, subset = rhs %in% c("class=e")) 

In [ ]:
inspect(head(x = rules.ap.class.edible, n = 25, by ="lift"))

In [ ]:
#::GMG::Ejemplo de las reglas de venenosas con más "lift"
rules.ap.class.poisonous <- subset(rules.ap.class, subset = rhs %in% c("class=p"))

In [ ]:
inspect(head(x = rules.ap.class.poisonous, n = 25, by ="lift"))

In [ ]:
inspect(rules.ap.class.poisonous[quality(rules.ap.class.poisonous)$support > 0.3])

#### Análisis gráfico

In [ ]:
plot(rules.ap.filtered, measure = c("support", "lift"), shading = "confidence")

In [ ]:
plot(rules.ap.filtered, method = "two-key plot", jitter = 0)

In [ ]:
plot(rules.ap.filtered[quality(rules.ap.filtered)$support > 0.3], method = "grouped")

In [ ]:
#plot(rules.ap.filtered[quality(rules.ap.filtered)$support > 0.3], method = 'paracoord',
#     control = list(reorder = TRUE))
plot(head(rules.ap.filtered, n = 25, by ='lift'), method = 'paracoord',
     control = list(reorder = TRUE))

In [ ]:
plot(head(rules.ap.filtered, n = 25, by = 'confidence'), method="graph")
#plot(rules.ap.filtered[quality(rules.ap.filtered)$support > 0.3], method="graph")


## B. Árboles de Decisión

### Punto 2 (4 puntos):

#### Bibliotecas

In [ ]:
library(caret)
library(tree)
library(rpart)
library(rpart.plot)

#### Dataset

En este apartado aplicaremos árboles de clasificación para obtener un modelo que permita clasificar una nueva entrada. Para ello, vamos a utilizaremos el paquete `CaReT`. Este paquete (y los demás que hemos visto para trabajar con árboles en `R`) no aceptan objetos del tipo `transactions` como los del apartado anterior. Por tanto, hemos preparado un fichero *csv* con el dataset *Mushrooms*; puedes descargarlo desde esta aquí:
https://github.com/SantanderMetGroup/Master-Data-Science/tree/master/Data_mining/datasets. Lee el dataset con la función `read.csv`.

**::GMG::** Yo utilizo el dataset preparado en mi **::GMG::Punto 0**.

In [ ]:
str(dataset_complete)

In [ ]:
table(dataset_complete$class)

In [ ]:
#summary(dataset_complete)

>Ahora ya tenemos un data.frame con el que podemos empezar a trabajar. En primer lugar tendremos que eliminar la columna 17 (`veil.type`), ya que contiene un único nivel y daría errores en `CaReT`(esta columna podría eliminarse también en el caso de las reglas de asociación ya que no aporta información al dataset). 

**::GMG::** Esto ya lo he tenido en cuenta en mi dataset (ver *::GMG::Punto 0**)

#### Rpart Train

Nuestro objetivo será *encontrar la configuración (profundidad) óptima del árbol*. 

Para ello, partiremos el dataset en dos subconjuntos indpendedientes de train y test (75% y 25% del total, respectivamente). 

In [ ]:
#::GMG::Hago la patición train/test del dataset tal y como nos indican
#       En clase vimos una partición basada en el uso de índices con sample()
set.seed(666) # Fijo un seed diabólico :)
#n <- nrow(dataset_complete)
#indtrain <- sample(1:n, 0.75 * n)
#indtest <- setdiff(1:n, indtrain)
#dataset.train <- dataset_complete[indtrain,]
#dataset.test <- dataset_complete[indtest,]
#::GMG::Una alternativa "lazy" ya que estamos usando "caret" es usar la función
#       createDataPartition para el train/test
# https://www.rdocumentation.org/packages/caret/versions/6.0-81/topics/createDataPartition
trainidx <- createDataPartition(dataset_complete$class, p=0.75, list=FALSE)
#::nota::el dataset de entrenamiento será el compuesto por los índices seleccionados
dataset.train <- dataset_complete[trainidx,]
#::nota: la parte de test será "el resto"
dataset.test <- dataset_complete[-trainidx,]

Sobre el dataset de train, aplicaremos una cross-validación con 3 folds y la repetiremos 50 veces (recuerda que los árboles son sensibles a la partición train/test que se considere). 

In [ ]:
#::GMG::Aplico el entrenamiento descrito con 3-fold cross-validation, 50 veces
#       Primero tengo que fijar el "procedimiento de entrenamiento" de Caret
#       'number'-folds, 'repeats' times
mi.trControl <- trainControl(method = "repeatedcv",  number=3, repeats=50, verboseIter=FALSE)

In [ ]:
#::GMG::Hago el entrenamiento
modelo.rpart <- train(class ~ ., 
                      data = dataset.train,
                      method = "rpart2",
                      trControl = mi.trControl,
                      tuneGrid = expand.grid(maxdepth = 1:10),
                      metric = 'Accuracy')

In [ ]:
#::GMG::Echo un vistazo a lo que se ha obtenido
#::nota::una profundidad óptima de 6 según la métrica "Accuracy"
modelo.rpart

In [ ]:
#::Más cosas que se pueden ver
# https://www.statmethods.net/advstats/cart.html
summary(modelo.rpart$results)

In [ ]:
#::GMG::Las variables tenidas en cuenta
# https://www.rdocumentation.org/packages/caret/versions/6.0-81/topics/varImp
#::nota::la visualización no es muy buena porque mezcla el nombre
#        de la variable y su valor, hay que tirar de la descripción en
# https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names
#        para aclararse :)
varImp(modelo.rpart)

In [ ]:
#::GMG::El modelo final de árbol obtenido, "dibujado" más o menos :)
rpart.plot(modelo.rpart$finalModel, type = 5,cex=0.6)

**::DUDA::** ¿Cómo se interpreta ese modelo en "reglas"?

Se necesita la siguiente información de la [descripción del dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.names):

<pre>7. Attribute Information: (classes: edible=e, poisonous=p)
     1. cap-shape:                bell=b,conical=c,convex=x,flat=f,
                                  knobbed=k,sunken=s
     2. cap-surface:              fibrous=f,grooves=g,scaly=y,smooth=s
     3. cap-color:                brown=n,buff=b,cinnamon=c,gray=g,green=r,
                                  pink=p,purple=u,red=e,white=w,yellow=y
     4. bruises?:                 bruises=t,no=f
     5. odor:                     almond=a,anise=l,creosote=c,fishy=y,foul=f,
                                  musty=m,none=n,pungent=p,spicy=s
     6. gill-attachment:          attached=a,descending=d,free=f,notched=n
     7. gill-spacing:             close=c,crowded=w,distant=d
     8. gill-size:                broad=b,narrow=n
     9. gill-color:               black=k,brown=n,buff=b,chocolate=h,gray=g,
                                  green=r,orange=o,pink=p,purple=u,red=e,
                                  white=w,yellow=y
    10. stalk-shape:              enlarging=e,tapering=t
    11. stalk-root:               bulbous=b,club=c,cup=u,equal=e,
                                  rhizomorphs=z,rooted=r,missing=?
    12. stalk-surface-above-ring: fibrous=f,scaly=y,silky=k,smooth=s
    13. stalk-surface-below-ring: fibrous=f,scaly=y,silky=k,smooth=s
    14. stalk-color-above-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    15. stalk-color-below-ring:   brown=n,buff=b,cinnamon=c,gray=g,orange=o,
                                  pink=p,red=e,white=w,yellow=y
    16. veil-type:                partial=p,universal=u
    17. veil-color:               brown=n,orange=o,white=w,yellow=y
    18. ring-number:              none=n,one=o,two=t
    19. ring-type:                cobwebby=c,evanescent=e,flaring=f,large=l,
                                  none=n,pendant=p,sheathing=s,zone=z
    20. spore-print-color:        black=k,brown=n,buff=b,chocolate=h,green=r,
                                  orange=o,purple=u,white=w,yellow=y
    21. population:               abundant=a,clustered=c,numerous=n,
                                  scattered=s,several=v,solitary=y
    22. habitat:                  grasses=g,leaves=l,meadows=m,paths=p,
                                  urban=u,waste=w,woods=d</pre>

De izquierda a derecha, de arriba a abajo del árbol:

1. reglas para "edible":
 - R_1 := odor=none.AND.spore-print-color=NOT.green
 - R_2 := odor=NOT.none.AND.bruises=TRUE.AND.odor=NOT.foul.AND.odor=NOT.pungent
 
2. reglas para "poisonous":
 - R_1 :=  ...

**::nota::** duda **no resuelta**

#### Rpart Test

In [ ]:
#::GMG::Pruebo el modelo en el conjunto reservado para test final y
#       visualizo el resultado con confusionMatrix() de caret
# https://www.rdocumentation.org/packages/caret/versions/6.0-81/topics/confusionMatrix
modelo.rpart.test <- predict(modelo.rpart, dataset.test)
confusionMatrix(modelo.rpart.test, dataset.test$class)

**::GMG::** Cometemos un error *importante* de tipo *falso positivo*, i.e hay 12 casos que se han predicho *edible* cuando son **poisonous**. Es un _error *Type I*_ que en nuestro dataset es **mortal**. Es más importante tener aquí un *cero* que tenerlo en el otro valor de la diagonal (*falso negativo*). 

**::DUDA::** ¿Qué se puede hacer para asegurar un cero en FP?

**::nota::** Duda **no resuelta**

#### Análisis

* ¿Cuál es la configuración óptima del árbol? ¿Hay alguna diferencia entre el árbol *completo* y el óptimo? ¿Por qué crees que ocurre esto?
* ¿Cuáles son las dos variables que mayor peso tienen a la hora de clasificar? Entrena un nuevo árbol considerando como predictores únicamente esas dos variables. ¿Qué resultados obtienes? 
* Entrena un nuevo árbol considerando como predictores cualesquiera otras dos variables que no sean las utilizadas en la pregunta anterior. ¿Cuál es el error de test de este árbol?

### Punto 3 (3 puntos):

Por un lado, las ramas del árbol pueden ser interpretadas como reglas de forma similar a las obtenidas por el algoritmo de reglas aplicado. Por ejemplo, en el caso del árbol obtenido con el dataset `Play Tennis` puede obtenerse las siguientes `reglas`: SI Outlook = Overcast -> Play Tennis = Yes ó SI (Outlook = Sunny) AND (Humidity = Normal) -> Play Tennis = Yes, cuya confianza asociada viene dada por la frecuencia relativa de cada caso en esa rama del árbol. Por otro lado, considerando las reglas que implican a nuestra variable objetivo tendríamos un `modelo` similar al dado por el árbol. Considerar y comparar ambas aproximaciones (p.e. ¿coinciden los antecedentes de las reglas? ¿alguna de las variables más frecuentes como antecedente en las reglas se corresponde con alguna de las variables con mayor capacidad de discriminación? etc.). 

**::GMG::** Este apartado le dejo *sin hacer*.

**::nota::** En algún momento me gustaría tener la solución para poder aprender de ella.